In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train= pd.read_csv('train.csv',index_col='PassengerId')

In [77]:
test = pd.read_csv('test.csv',index_col='PassengerId')

In [3]:
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
train['Cabin'].isna().value_counts()

True     687
False    204
Name: Cabin, dtype: int64

with this amount of na values we cannot use or replace properly values, we need to drop it.

Embarked values are C = Cherbourg, Q = Queenstown, S = Southampton. Let's change to. 1 = Cherbourg, 2 = Queenstown, 3 = Southampton.

We can use the mean famele age and mean male age for the np.nan data in column Age

Let´s use another replace in the Sex columns, 1 if male and 0, if female.

We need to change the scale of Age and Fare.

Let's create a function to do all of this work

In [6]:
from sklearn.preprocessing import StandardScaler

In [7]:
scalar = StandardScaler()

In [51]:
def transform(data):
    data.drop(columns=['Cabin'],inplace=True)
    data.dropna(subset=['Embarked'],inplace=True) # dropped 2 values
    data['Embarked'].replace(['C','Q','S'],[1,2,3],inplace=True)
    data.loc[data['Sex']=='female','Age']=data.loc[data['Sex']=='female','Age'].fillna(np.mean(data.loc[data['Sex']=='female','Age']))
    data.loc[data['Sex']=='male','Age']=data.loc[data['Sex']=='male','Age'].fillna(np.mean(data.loc[data['Sex']=='male','Age']))
    data['Sex'].replace({'male':1,'female':0}, inplace=True)
    data['Age'] = scalar.fit_transform(np.array(data['Age']).reshape(-1,1))
    data['Fare'] = scalar.fit_transform(np.array(data['Fare']).reshape(-1,1))
    b = []
    for i in data['Ticket']:
        a = i.split(' ')
        try:
            b.append(float(a[len(a)-1]))
        except:
            b.append(np.nan)
    data['Ticket_num'] = b
    data['Ticket_num'] = data['Ticket_num'].fillna(data['Ticket_num'].mean()) #sub 4 NA values
    data['Ticket_num'] = scalar.fit_transform(np.array(data['Ticket_num']).reshape(-1,1))
    return data

In [9]:
train_trans = transform(train)

In [10]:
train_trans.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Ticket_num
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",1,-0.591902,1,0,A/5 21171,-0.500240,3,-0.422846
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,0.641182,1,0,PC 17599,0.788947,1,-0.428287
3,1,3,"Heikkinen, Miss. Laina",0,-0.283631,0,0,STON/O2. 3101282,-0.486650,3,4.269261
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,0.409979,1,0,113803,0.422861,3,-0.281734
5,0,3,"Allen, Mr. William Henry",1,0.409979,0,0,373450,-0.484133,3,0.113801


In [11]:
train_trans.describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Ticket_num
count,889.000000,889.000000,889.000000,8.890000e+02,889.000000,889.000000,8.890000e+02,889.000000,8.890000e+02
mean,0.382452,2.311586,0.649044,-3.410595e-16,0.524184,0.382452,1.092115e-16,2.535433,-2.713115e-17
std,0.486260,0.834700,0.477538,1.000563e+00,1.103705,0.806761,1.000563e+00,0.792088,1.000563e+00
min,0.000000,1.000000,0.000000,-2.255025e+00,0.000000,0.000000,-6.462044e-01,1.000000,-4.550920e-01
25%,0.000000,2.000000,0.000000,-5.919021e-01,0.000000,0.000000,-4.872378e-01,2.000000,-4.324305e-01
50%,0.000000,3.000000,1.000000,2.464018e-02,0.000000,0.000000,-3.551972e-01,3.000000,-2.839033e-01
75%,1.000000,3.000000,1.000000,4.099791e-01,1.000000,0.000000,-2.207954e-02,3.000000,7.363303e-02
max,1.000000,3.000000,1.000000,3.878029e+00,8.000000,6.000000,9.668551e+00,3.000000,4.269314e+00


In [12]:
X_train, y_train = train_trans[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Ticket_num']], train_trans['Survived']

let's fit some model and then combine then into an Ensemble.

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
log_clf = LogisticRegression()
log_clf.fit(X_train,y_train)

LogisticRegression()

In [15]:
y_pred = log_clf.predict(X_train)

In [16]:
accuracy_score(y_train,y_pred)

0.8008998875140607

In [17]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [18]:
sgd_clf = SGDClassifier()
paramns = {'max_iter': range(200,500)}
grid_clf = GridSearchCV(sgd_clf, paramns, n_jobs=-1, cv=5)
grid_clf.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SGDClassifier(), n_jobs=-1,
             param_grid={'max_iter': range(200, 500)})

In [19]:
grid_clf.best_params_

{'max_iter': 312}

In [20]:
y_pred = grid_clf.best_estimator_.predict(X_train)
accuracy_score(y_train,y_pred)

0.7851518560179978

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
knn_clf = KNeighborsClassifier()
paramns = {'n_neighbors': range(30,50)}
grid_clf1 = GridSearchCV(knn_clf, paramns, n_jobs=-1, cv=5)
grid_clf1.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': range(30, 50)})

In [23]:
grid_clf1.best_params_

{'n_neighbors': 31}

In [24]:
y_pred = grid_clf1.best_estimator_.predict(X_train)
accuracy_score(y_train,y_pred)

0.8155230596175478

In [25]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()

In [26]:
paramns = {'max_depth': range(50)}
grid_clf3 = GridSearchCV(tree_clf, paramns, cv=5, n_jobs=-1)
grid_clf3.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 306, in fit
    raise ValueError("max_depth must be greater than zero. ")
ValueError: max_depth must be greater than

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': range(0, 50)})

In [27]:
grid_clf3.best_params_

{'max_depth': 7}

In [28]:
y_pred = grid_clf3.best_estimator_.predict(X_train)
accuracy_score(y_train, y_pred)

0.889763779527559

In [29]:
from sklearn.ensemble import RandomForestClassifier

In [30]:
rnd_clf = RandomForestClassifier(n_estimators = 1000, max_leaf_nodes = 16, n_jobs =-1)

In [31]:
rnd_clf.fit(X_train,y_train)

RandomForestClassifier(max_leaf_nodes=16, n_estimators=1000, n_jobs=-1)

In [32]:
y_pred = rnd_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.859392575928009

In [33]:
for name, score in zip(X_train.columns, rnd_clf.feature_importances_):
    print(name,score)

Pclass 0.12387591806272352
Sex 0.42556059754765985
Age 0.09651844048159357
SibSp 0.03634507339321516
Parch 0.026595989372296572
Fare 0.15105813544513974
Embarked 0.02626599105659813
Ticket_num 0.11377985464077345


In [34]:
from sklearn.ensemble import VotingClassifier

In [36]:
named_estimators = [
    ('rnd_clf', rnd_clf),
    ('grid_clf3', grid_clf3.best_estimator_),
    ('grid_clf1', grid_clf1.best_estimator_),
    ('grid_clf', grid_clf.best_estimator_),
    ('log_clf', log_clf)
]

In [39]:
voting_clf = VotingClassifier(named_estimators)

In [40]:
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rnd_clf',
                              RandomForestClassifier(max_leaf_nodes=16,
                                                     n_estimators=1000,
                                                     n_jobs=-1)),
                             ('grid_clf3', DecisionTreeClassifier(max_depth=7)),
                             ('grid_clf1',
                              KNeighborsClassifier(n_neighbors=31)),
                             ('grid_clf', SGDClassifier(max_iter=312)),
                             ('log_clf', LogisticRegression())])

In [41]:
y_pred = voting_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.8526434195725534

In [46]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [78]:
test_trans = transform(test)

In [79]:
test_trans.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Ticket_num
PassengerId,,,,,,,,,,
892,3,"Kelly, Mr. James",1,0.334992,0,0,330911,-0.497811,2,0.132042
893,3,"Wilkes, Mrs. James (Ellen Needs)",0,1.325529,1,0,363272,-0.512660,3,0.187173
894,2,"Myles, Mr. Thomas Francis",1,2.514174,0,0,240276,-0.464532,2,-0.022366
895,3,"Wirz, Mr. Albert",1,-0.259330,0,0,315154,-0.482888,3,0.105198
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,-0.655545,1,1,3101298,-0.417971,3,4.851741


In [80]:
test_trans.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Ticket_num
count,418.000000,418.000000,4.180000e+02,418.000000,418.000000,4.170000e+02,418.000000,4.180000e+02
mean,2.265550,0.636364,1.279070e-16,0.447368,0.392344,-5.258251e-17,2.401914,-1.646742e-17
std,0.841838,0.481622,1.001198e+00,0.896760,0.981429,1.001201e+00,0.854496,1.001198e+00
min,1.000000,0.000000,-2.385420e+00,0.000000,0.000000,-6.380170e-01,1.000000,-4.317024e-01
25%,1.000000,0.000000,-5.763024e-01,0.000000,0.000000,-4.966178e-01,2.000000,-4.086933e-01
50%,3.000000,1.000000,-1.863304e-05,0.000000,0.000000,-3.791690e-01,3.000000,-3.710406e-01
75%,3.000000,1.000000,4.340458e-01,1.000000,0.000000,-7.391031e-02,3.000000,1.568968e-01
max,3.000000,1.000000,3.623576e+00,8.000000,9.000000,8.536851e+00,3.000000,4.851770e+00


In [81]:
test_trans['Fare'] = test_trans['Fare'].fillna(test_trans['Fare'].mean())

In [82]:
test_trans.describe()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Ticket_num
count,418.000000,418.000000,4.180000e+02,418.000000,418.000000,4.180000e+02,418.000000,4.180000e+02
mean,2.265550,0.636364,1.279070e-16,0.447368,0.392344,-5.245671e-17,2.401914,-1.646742e-17
std,0.841838,0.481622,1.001198e+00,0.896760,0.981429,1.000000e+00,0.854496,1.001198e+00
min,1.000000,0.000000,-2.385420e+00,0.000000,0.000000,-6.380170e-01,1.000000,-4.317024e-01
25%,1.000000,0.000000,-5.763024e-01,0.000000,0.000000,-4.966178e-01,2.000000,-4.086933e-01
50%,3.000000,1.000000,-1.863304e-05,0.000000,0.000000,-3.791690e-01,3.000000,-3.710406e-01
75%,3.000000,1.000000,4.340458e-01,1.000000,0.000000,-7.391031e-02,3.000000,1.568968e-01
max,3.000000,1.000000,3.623576e+00,8.000000,9.000000,8.536851e+00,3.000000,4.851770e+00


In [83]:
X_test = test_trans[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked','Ticket_num']]

In [84]:
y_pred = voting_clf.predict(X_test)

In [85]:
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [86]:
df = pd.DataFrame({'Survived':y_pred}, index = test_trans.index)

In [87]:
df.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [88]:
df.to_csv('resultado.csv')

In [89]:
import pickle

In [90]:
with open('1modelo_titanic.pkl','wb' ) as file:
    pickle.dump(voting_clf, file)